Ce code construit une stratégie “pilotée par un modèle” sur un panier de cryptos (BTC, ETH, BNB, SOL, XRP, DOGE).

L’idée est de partir d’une allocation de base low-vol (on privilégie les actifs les moins volatils), puis d’entraîner deux modèles de régression logistique qui estiment la probabilité d’un mouvement extrême le lendemain (forte baisse ou forte hausse). 

Sur la fenêtre de test (les 365 derniers jours), le modèle choisit chaque jour un régime parmi trois : baseline (low-vol), risk-on (equal-weight) ou risk-off (cash). 

Ensuite on backteste cette allocation “adaptative” et on la compare à la stratégie low-vol de base, en tenant compte de coûts de transaction via le turnover.

In [ ]:
import pandas as pd
import numpy as np

# params
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "DOGEUSDT"]
DATA_DIR = "binance_public_data"
START = "2018-01-01"
END   = "2025-12-31"

WIN_VOL = 20
LAM = 0.94
WIN_MOM1 = 5
WIN_MOM2 = 20
WIN_MOM3 = 60
Q_MOVE = 0.3          # quantile tail
TEST_DAYS = 365
FREQ = 365
COST_BPS = 10

# load
def load_symbol(sym):
    fn = f"{DATA_DIR}/{sym}_1d_2021_2025.csv"  # adapte si fichier 2018-2025
    df = pd.read_csv(fn, parse_dates=["timestamp"], index_col="timestamp").sort_index()
    df = df.loc[START:END]
    df["ret"] = np.log(df["close"] / df["close"].shift(1))
    return df.dropna(subset=["ret"])

dfs = {s: load_symbol(s) for s in SYMBOLS}

# align
idx = None
for s in SYMBOLS:
    idx = dfs[s].index if idx is None else idx.intersection(dfs[s].index)

rets = pd.DataFrame({s: dfs[s].loc[idx, "ret"] for s in SYMBOLS}).dropna()
close = pd.DataFrame({s: dfs[s].loc[rets.index, "close"] for s in SYMBOLS})
high  = pd.DataFrame({s: dfs[s].loc[rets.index, "high"] for s in SYMBOLS})
low   = pd.DataFrame({s: dfs[s].loc[rets.index, "low"] for s in SYMBOLS})
openp = pd.DataFrame({s: dfs[s].loc[rets.index, "open"] for s in SYMBOLS})
volu  = pd.DataFrame({s: dfs[s].loc[rets.index, "volume"] for s in SYMBOLS})

# vol ewma
vol_ewma = np.sqrt((rets**2).ewm(alpha=1-LAM).mean())
vol_roll = rets.rolling(WIN_VOL).std()

# base weights low_vol
q_cs = vol_roll.quantile(0.5, axis=1)
w_low = (vol_roll.le(q_cs, axis=0)).astype(float)

# features per asset -> stacked
feat = {}

# momentum
feat["ret_1"]  = rets
feat["ret_5"]  = close.pct_change(WIN_MOM1)
feat["ret_20"] = close.pct_change(WIN_MOM2)
feat["ret_60"] = close.pct_change(WIN_MOM3)

# vol
feat["vol_ewma"] = vol_ewma
feat["vol_roll20"] = vol_roll
feat["vol_ratio"] = vol_ewma / (vol_ewma.rolling(252).mean())

# range body
range_ = (high - low).replace(0, np.nan)
body_ = (close - openp).abs()
feat["body_ratio"] = (body_ / range_)

# volume
feat["vol_norm"] = volu / volu.rolling(20).mean()

# stack
X = []
for k, mat in feat.items():
    tmp = mat.stack().rename(k)
    X.append(tmp)

X = pd.concat(X, axis=1).dropna()

# future return for label (t+1)
fut = rets.shift(-1).stack().rename("ret_fut")
df_all = pd.concat([X, fut], axis=1).dropna()

# labels tails
down_thr = df_all["ret_fut"].quantile(Q_MOVE)
up_thr   = df_all["ret_fut"].quantile(1 - Q_MOVE)

df_all["y_down"] = (df_all["ret_fut"] <= down_thr).astype(int)
df_all["y_up"]   = (df_all["ret_fut"] >= up_thr).astype(int)

# train/test time
cutoff = df_all.index.get_level_values(0).max() - pd.Timedelta(days=TEST_DAYS)

train = df_all[df_all.index.get_level_values(0) < cutoff].copy()
test  = df_all[df_all.index.get_level_values(0) >= cutoff].copy()

print("dates train:", train.index.get_level_values(0).min(), "->", train.index.get_level_values(0).max(), "n", len(train))
print("dates test :", test.index.get_level_values(0).min(),  "->", test.index.get_level_values(0).max(),  "n", len(test))
print("down thr:", float(down_thr), "up thr:", float(up_thr))
print("share down:", float(train["y_down"].mean()), "share up:", float(train["y_up"].mean()))


dates train: 2021-09-10 00:00:00 -> 2024-12-29 00:00:00 n 7242

dates test : 2024-12-30 00:00:00 -> 2025-12-30 00:00:00 n 2196

down thr: -0.013790305260461067 up thr: 0.013418659778213805

share down: 0.2972935653134493 share up: 0.3036454018227009